In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import pickle

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
path_corpus = Path('../../data/corpora/tutby_126784.csv')
path_train_split = Path('../../data/corpora/ttsplit.pickle')

In [3]:
data = pd.read_csv(path_corpus)

data['tags'] = data['tags'].apply(eval)
data['tags'] = data['tags'].apply(lambda tags: [tag.lower().strip() for tag in tags])

print(data.shape)
display(data.head(5))

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,Главный тренер солигорского «Шахтера» Олег Куб...,[футбол]
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,Планы по восстановлению рисунка есть. Но пока ...,"[архитектура, живопись, жкх]"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,Областное управление МЧС ко Дню пожарной служб...,[министерства]
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,Чемпион Беларуси БАТЭ воспользовался паузой в ...,"[футбол, батэ]"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,Генпрокуратура Украины откроет уголовное произ...,"[ситуация в украине, государственные перевороты]"


In [4]:
tags_sorted = data['tags'].explode().value_counts().index.tolist()
print(len(tags_sorted))

def select_tag(tags):
    return min(tags, key=lambda tag: tags_sorted.index(tag))

data_tags = data['tags'].map(select_tag)

print(data_tags.value_counts().shape)
data_tags.head(5)

1278
(687,)


0                футбол
1           архитектура
2          министерства
3                футбол
4    ситуация в украине
Name: tags, dtype: object

In [5]:
counts = data_tags.value_counts(normalize=True)
index = counts.cumsum() <= 0.95
common_tags = counts[index].index.tolist()

data_with_tag_index = data_tags.isin(common_tags)
data_tags = data_tags[data_with_tag_index]

print(data_tags.value_counts().shape)

(138,)


In [6]:
# label_binarizer = MultiLabelBinarizer()
label_binarizer = LabelBinarizer()

# data_tags = data['tags']
y = label_binarizer.fit_transform(data_tags)

print(len(label_binarizer.classes_))
print(y.shape)

138
(120444, 138)


In [7]:
index = np.arange(len(y))

train_index, test_index = train_test_split(index, test_size=0.33, stratify=y)

print(train_index.shape)
print(test_index.shape)

(80697,)
(39747,)


In [8]:
with open(path_train_split, 'wb') as file:
    pickle.dump((data_with_tag_index, train_index, test_index, y, label_binarizer), file)

with open(path_train_split, 'rb') as file:
    data_with_tag_index, train_index, test_index, y, label_binarizer = pickle.load(file)